#### Task

Write a python class called PinholeCamera, the class should have the following methods:

 1- Constructor
def __init__(self, K : np.ndarray ,dist_coefs : np.ndarray, world_R_cam : np.ndarray  , world_t_cam : np.ndarray )
  """
  constructor, that takes camera intrinsic and extrinsics
  """

2 - Project
def project(self, world_pts_3d : np.ndarray) -> np.ndarray
  """
    projects a 3d array of points into camera plane
  """

3 - Unproject

def unproject(self, image_pts_2d : np.ndarray) -> np.ndarray
  """
    projects 2d detected points from the image as camera rays
  """

write tests (with data that you create) that show that your class  works properly

In [ ]:
import numpy as np
import cv2

class PinholeCamera:
    def __init__(self, K: np.ndarray, dist_coefs: np.ndarray, R: np.ndarray, t: np.ndarray):
        self.K = K
        self.dist_coefs = dist_coefs
        self.R = R
        self.t = t.reshape(-1, 1)  # Ensure t is a 3x1 column vector (or 3x1 row vector)

    def project(self, world_pts_3d: np.ndarray) -> np.ndarray:
        # Ensure world_pts_3d is in the correct shape and type
        world_pts_3d = np.asarray(world_pts_3d, dtype=np.float32).reshape(-1, 3)
        
        # Project the 3D points into the 2D image plane using OpenCV's projectPoints
        image_pts, _ = cv2.projectPoints(world_pts_3d, self.R, self.t, self.K, self.dist_coefs)
        
        return image_pts.squeeze()

    def unproject(self, image_pts_2d: np.ndarray) -> np.ndarray:
        # Undistort the image points before unprojection
        image_pts_2d = cv2.undistortPoints(image_pts_2d.reshape(-1, 1, 2), self.K, self.dist_coefs)
        
        rays = image_pts_2d.reshape(-1, 2)
        ones = np.ones((rays.shape[0], 1))
        cam_rays = np.hstack((rays, ones))
        
        # Rotate to world frame
        return cam_rays @ self.R.T  


In [2]:
def test_pinhole_camera():
    K = np.array([[800, 0, 320],
                  [0, 800, 240],
                  [0, 0, 1]], dtype=np.float32)
    dist = np.zeros(5, dtype=np.float32)
    R = np.eye(3, dtype=np.float32)
    t = np.array([[0], [0], [-5]], dtype=np.float32)
    
    cam = PinholeCamera(K, dist, R, t)
    
    world_pts = np.array([[0, 0, 0],
                          [1, 1, 0],
                          [-1, -1, 0]], dtype=np.float32)
    
    proj = cam.project(world_pts)
    print("Projected points:\n", proj)

test_pinhole_camera()


Projected points:
 [[320. 240.]
 [160.  80.]
 [480. 400.]]
